# Project: Creating Relations In SQLite
The Academy Awards, also known as the Oscars, is an annual awards ceremony hosted to recognize the achievements in the film industry. There are many different awards categories and the members of the academy vote every year to decide which artist or film should get the award. Each row in our data represents a nomination for an award. The database file, nominations.db, contains just the nominations table.
## Exploring The Database

In [14]:
import sqlite3
conn = sqlite3.connect('nominations.db')
schema = conn.execute("pragma table_info(nominations);").fetchall()
first_ten = conn.execute("select * from nominations limit 10;").fetchall()

for r in schema:
    print(r)
for r in first_ten:
    print(r)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)
(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting 

This table has the following schema:

- Year - the year of the awards ceremony, integer type.
- Category - the category of award the nominee was nominated for, text type.
- Nominee - the person nominated for the award, text type.
- Movie - the movie the nominee participated in, text type.
- Character - the name of the character the nominee played, text type.
- Won - if this nominee won the award, integer type.

## Creating The Ceremonies Table
Let's create a ceremonies table that contains the Year and Host for each ceremony and then set up a one-to-many relationship between ceremonies and nominations. In this screen, we'll focus on creating the ceremonies table and inserting the data we need and in the next guided step, we'll focus on setting up the one-to-many relationship.

The ceremonies table will contain 3 fields:

- id - unique identifier for each row, integer type.
- Year - the year of the awards ceremony, integer type.
- Host - the host of the awards ceremony, text type.

Before we can create and insert into the ceremonies table, we need to look up the host for each ceremony from 2000 to 2010. The Python sqlite3 library comes with an executemany method that let's us easily mass insert records into a table. The executemany method requires the records we want to insert to be represented as a list of tuples. We then just need to write a single INSERT query with placeholder elements and specify that we want the list of tuples to be dropped into the query.

In [15]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]
create_ceremonies = "create table ceremonies (id integer primary key, year integer, host text);"
conn.execute(create_ceremonies)
insert_query = "insert into ceremonies (Year, Host) values (?,?);"
conn.executemany(insert_query, years_hosts)

conn.execute("select * from ceremonies limit 10;").fetchall()

[(1, 2010, 'Steve Martin'),
 (2, 2009, 'Hugh Jackman'),
 (3, 2008, 'Jon Stewart'),
 (4, 2007, 'Ellen DeGeneres'),
 (5, 2006, 'Jon Stewart'),
 (6, 2005, 'Chris Rock'),
 (7, 2004, 'Billy Crystal'),
 (8, 2003, 'Steve Martin'),
 (9, 2002, 'Whoopi Goldberg'),
 (10, 2001, 'Steve Martin')]

In [16]:
conn.execute("pragma table_info(ceremonies);").fetchall()

[(0, 'id', 'integer', 0, None, 1),
 (1, 'year', 'integer', 0, None, 0),
 (2, 'host', 'text', 0, None, 0)]

## Foreign Key Constraints
Since we'll be creating relations using foreign keys, we need to turn on foreign key constraints. By default, if we insert a row into a table that contains one or multiple foreign key columns, the record will be successfully inserted even if the foreign key reference is incorrect.

For example, since the ceremonies table only contains the id values 1 to 10, inserting a row into nominations while specifying that the ceremony_id value be 11 will work and no error will be returned. This is problematic because if we try to actually join that row with the ceremonies table, the results set will be empty since the id value 11 doesn't map to any row in the ceremonies table.

In [17]:
conn.execute("PRAGMA foreign_keys = ON;")

## Setting Up One-To-Many
The next step is to remove the Year column from nominations and add a new column, ceremony_id, that contains the foreign key reference to the id column in the ceremonies table. Unfortunately, we can't remove columns from an existing table in SQLite or change its schema. The goal of SQLite is to create an incredibly lightweight, open source database that contains a common, but reduced, set of features. While this has allowed SQLite to become the most popular database in the world, SQLite doesn't have the ability to heavily modify an existing table to keep the code base lightweight.

The only alterations we can make to an existing table are renaming it or adding a new column. This means that we can't just remove the Year column from nominations and add the ceremony_id column. 

In [18]:
create_nominations_two = '''create table nominations_two 
(id integer primary key, 
category text, 
nominee text, 
movie text, 
character text, 
won text,
ceremony_id integer,
foreign key(ceremony_id) references ceremonies(id));
'''

nom_query = '''
select ceremonies.id as ceremony_id, nominations.category as category, 
nominations.nominee as nominee, nominations.movie as movie, 
nominations.character as character, nominations.won as won
from nominations
inner join ceremonies 
on nominations.year == ceremonies.year
;
'''
joined_nominations = conn.execute(nom_query).fetchall()

conn.execute(create_nominations_two)

insert_nominations_two = '''insert into nominations_two (ceremony_id, category, nominee, movie, character, won) 
values (?,?,?,?,?,?);
'''

conn.executemany(insert_nominations_two, joined_nominations)
conn.execute("select * from nominations_two limit 5;").fetchall()

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', '0', 1),
 (2,
  'Actor -- Leading Role',
  'Jeff Bridges',
  'True Grit',
  'Rooster Cogburn',
  '0',
  1),
 (3,
  'Actor -- Leading Role',
  'Jesse Eisenberg',
  'The Social Network',
  'Mark Zuckerberg',
  '0',
  1),
 (4,
  'Actor -- Leading Role',
  'Colin Firth',
  "The King's Speech",
  'King George VI',
  '1',
  1),
 (5,
  'Actor -- Leading Role',
  'James Franco',
  '127 Hours',
  'Aron Ralston',
  '0',
  1)]

## Deleting And Renaming Tables
We now need to delete the nominations table since we'll be using the nominations_two table moving forward. We can use the DROP TABLE statement to drop the original nominations table.

In [19]:
drop_nominations = "drop table nominations;"
conn.execute(drop_nominations)

rename_nominations_two = "alter table nominations_two rename to nominations;"
conn.execute(rename_nominations_two)

## Creating A Join Table

In [20]:
create_movies = "create table movies (id integer primary key,movie text);"
create_actors = "create table actors (id integer primary key,actor text);"
create_movies_actors = '''create table movies_actors (id INTEGER PRIMARY KEY,
movie_id INTEGER references movies(id), actor_id INTEGER references actors(id));
'''
conn.execute(create_movies)
conn.execute(create_actors)
conn.execute(create_movies_actors)

## Extra Exploration Ideas
- What other datasets can we add to the database?
- Based on what we know, brainstorm how we would populate the join table and the linked tables using data from nominations.